In [ ]:
import tweepy
import numpy as np
import pandas as pd


import signal
import sys
import time
import threading
import re
import string





# chiavi tokens
consumer_key = "JQPmCCOJjMVNvAyJUC3oSX761"
consumer_secret = "odPPjeMnHz2oLmAfBwfm6BefIVftL6sTtbxSATJ5ZV0zYUiWUb"
access_token = "1362405877702152192-8hsypanPfvL3zj3unzv72HGZEM1SlI"
access_token_secret = "UPwfdUIK5mK280qze8yPlYNr4B2vVYh6ri1i5G6JAW4w2"

# StreamListener class
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status.id_str)
      
      # verifica retweet
        is_retweet = hasattr(status, "retweeted_status")

        # verifica se il testo è troncato
        if hasattr(status,"extended_tweet"):
            text = status.extended_tweet["full_text"]
        else:
            text = status.text

        # verifica se presente citazione
        is_quote = hasattr(status, "quoted_status")
        quoted_text = ""
        if is_quote:
            # check if quoted tweet's text has been truncated before recording it
            if hasattr(status.quoted_status,"extended_tweet"):
                quoted_text = status.quoted_status.extended_tweet["full_text"]
            else:
                quoted_text = status.quoted_status.text

        # rimuove aratteri che potrebbero creare problemi con csv encoding
        remove_characters = ("\n","\t",",")
        for c in remove_characters:
            text=text.replace(c," ")
            quoted_text=quoted_text.replace(c," ")

        #remove special characters (alla fine rimossi tutti =))
        words = re.split(r"\W+", text)
            

        with open("sanremo.json", "a", encoding='utf-16') as f:
            f.write("%s,%s,%s,%s,%s,%s\n" % (status.created_at,status.user.screen_name,is_retweet,is_quote,text,quoted_text))

    def on_error(self, status_code):
        print("Encountered streaming error (", status_code, ")")
        sys.exit()

if __name__ == "__main__":
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)

    
producer = KafkaProducer(
  #bootstrap_servers=["kafka:9092"],
  #value_serializer=lambda v: json.dumps(v, indent=4, sort_keys=True, default=str).encode("utf-8"))

listener = MyListener(producer=producer, producer_topic="sanremotweet")

    # initialize stream
    stream = tweepy.Stream(auth=api.auth, listener=listener, tweet_mode='extended')
    with open("sanremo.json", "w", encoding='utf-16') as f:
        
        f.write("date,user,is_retweet,is_quote,text,quoted_text\n")
        
    tags = ["#Sanremo2021"]
    stream.filter(track=tags)


